In [1]:
import torch
import numpy as np
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision.datasets import MNIST
from skorch import NeuralNetClassifier
from modAL.models import ActiveLearner
from batchbald_redux import batchbald
from acquisition_functions import *
import os
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from keras.layers import Input, Dense, Lambda, Layer
from keras.models import Model
from keras import backend as K
import keras
from keras.datasets import cifar10

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
LATENT_DIMS = [256]
latent_dim = 256
MAX_EPOCHS = 200
BATCH_SIZE = 128
LEARNING_RATE = 0.001
EXPERIMENT_COUNT = 3
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 512
ORIGINAL_DIM = 784
results_path = 'results\\cifar10_infovae_dbal'
epsilon_std = 1.0

if not os.path.exists(results_path):
    os.makedirs(results_path)

ACQ_FUNCS = {
    "var_ratios": var_ratios,
    "mean_std": mean_std,
    "max_entropy": max_entropy,
    "bald": bald,
    "uniform": uniform,
#     "batch_bald": batch_bald
}


In [4]:
class MLP_REG(nn.Module):
    def __init__(self, latent_dim):
        super(MLP_REG, self).__init__()
        self.layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(latent_dim, 512),
            nn.ReLU(),
            nn.Dropout(p=0.25),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(p=0.25),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(128, 10),)
        
        
    def forward(self, x):
        return self.layers(x)


### read training data

In [5]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X_train = (X_train.astype(np.float32)) / 255.
X_train = X_train.reshape(X_train.shape[0], -1)

X_test = (X_test.astype(np.float32)) / 255.
X_test = X_test.reshape(X_test.shape[0], -1)

In [6]:
y_train = y_train.reshape((y_train.shape[0], ))
y_test = y_test.reshape((y_test.shape[0], ))

In [7]:
def active_learning_procedure(query_strategy,
                              X_test,
                              y_test,
                              X_pool,
                              y_pool,
                              X_initial,
                              y_initial,
                              estimator,
                              n_queries=98,
                              n_instances=10):
    learner = ActiveLearner(estimator=estimator,
                            X_training=X_initial,
                            y_training=y_initial,
                            query_strategy=query_strategy,
                           )
    perf_hist = [learner.score(X_test, y_test)]
    active_pool_size = [len(X_initial)]
    pool_size = len(X_initial)
    for index in range(n_queries):
        query_idx, query_instance = learner.query(X_pool, n_instances)
        learner.teach(X_pool[query_idx], y_pool[query_idx])
        X_pool = np.delete(X_pool, query_idx, axis=0)
        y_pool = np.delete(y_pool, query_idx, axis=0)
        model_accuracy = learner.score(X_test, y_test)
        pool_size = pool_size + n_instances
        print('Accuracy after query {n}: {acc:0.4f}'.format(n=index + 1, acc=model_accuracy))
        perf_hist.append(model_accuracy)
        active_pool_size.append(pool_size)
    return perf_hist, active_pool_size

In [8]:
X_train_enhanced = np.load(r"C:\Users\pinar\OneDrive\Masaüstü\masterthesis\src\Generative Models\cifar10\x_train_enhanced_infovae.npy")
X_test_enhanced = np.load(r"C:\Users\pinar\OneDrive\Masaüstü\masterthesis\src\Generative Models\cifar10\x_test_enhanced_infovae.npy")

for exp_iter in range(EXPERIMENT_COUNT):
    np.random.seed(exp_iter)
    initial_idx = np.array([],dtype=int)
    for i in range(10):
        idx = np.random.choice(np.where(y_train==i)[0], size=2, replace=False)
        initial_idx = np.concatenate((initial_idx, idx))

    for func_name, acquisition_func in ACQ_FUNCS.items():  

        X_initial = X_train_enhanced[initial_idx]
        y_initial = y_train[initial_idx]

        X_pool = np.delete(X_train_enhanced, initial_idx, axis=0)
        y_pool = np.delete(y_train, initial_idx, axis=0)

        model = MLP_REG(latent_dim).to(DEVICE)

        estimator = NeuralNetClassifier(model,
                                      max_epochs=MAX_EPOCHS,
                                      batch_size=BATCH_SIZE,
                                      lr=LEARNING_RATE,
                                      optimizer=torch.optim.Adam,
                                      criterion=torch.nn.CrossEntropyLoss,
                                      train_split=None,
                                      verbose=0,
                                      device=DEVICE)



        acc_arr, dataset_size_arr = active_learning_procedure(acquisition_func,
                                                          X_test_enhanced,
                                                          y_test,
                                                          X_pool,
                                                          y_pool,
                                                          X_initial,
                                                          y_initial,
                                                          estimator,)
        file_name = os.path.join(results_path, "{func_name}_latent_dim_{latent_dim}_exp_{exp_iter}.npy".format(func_name=func_name, exp_iter=exp_iter, latent_dim=latent_dim))
        np.save(file_name, (acc_arr, dataset_size_arr))


Accuracy after query 1: 0.1768
Accuracy after query 2: 0.1758
Accuracy after query 3: 0.1793
Accuracy after query 4: 0.1869
Accuracy after query 5: 0.1883
Accuracy after query 6: 0.1939
Accuracy after query 7: 0.2046
Accuracy after query 8: 0.2087
Accuracy after query 9: 0.2123
Accuracy after query 10: 0.2165
Accuracy after query 11: 0.2141
Accuracy after query 12: 0.2163
Accuracy after query 13: 0.2186
Accuracy after query 14: 0.2207
Accuracy after query 15: 0.2216
Accuracy after query 16: 0.2240
Accuracy after query 17: 0.2334
Accuracy after query 18: 0.2410
Accuracy after query 19: 0.2398
Accuracy after query 20: 0.2487
Accuracy after query 21: 0.2473
Accuracy after query 22: 0.2528
Accuracy after query 23: 0.2576
Accuracy after query 24: 0.2521
Accuracy after query 25: 0.2518
Accuracy after query 26: 0.2569
Accuracy after query 27: 0.2561
Accuracy after query 28: 0.2601
Accuracy after query 29: 0.2548
Accuracy after query 30: 0.2616
Accuracy after query 31: 0.2718
Accuracy after qu

Accuracy after query 61: 0.2914
Accuracy after query 62: 0.2923
Accuracy after query 63: 0.2908
Accuracy after query 64: 0.3070
Accuracy after query 65: 0.3025
Accuracy after query 66: 0.3024
Accuracy after query 67: 0.2978
Accuracy after query 68: 0.3040
Accuracy after query 69: 0.3021
Accuracy after query 70: 0.3041
Accuracy after query 71: 0.3050
Accuracy after query 72: 0.3098
Accuracy after query 73: 0.3133
Accuracy after query 74: 0.3129
Accuracy after query 75: 0.3013
Accuracy after query 76: 0.3082
Accuracy after query 77: 0.3100
Accuracy after query 78: 0.3154
Accuracy after query 79: 0.3114
Accuracy after query 80: 0.3079
Accuracy after query 81: 0.3127
Accuracy after query 82: 0.3100
Accuracy after query 83: 0.3150
Accuracy after query 84: 0.3085
Accuracy after query 85: 0.3117
Accuracy after query 86: 0.3166
Accuracy after query 87: 0.3141
Accuracy after query 88: 0.3087
Accuracy after query 89: 0.3144
Accuracy after query 90: 0.3129
Accuracy after query 91: 0.3143
Accuracy

Accuracy after query 23: 0.2802
Accuracy after query 24: 0.2904
Accuracy after query 25: 0.2938
Accuracy after query 26: 0.2791
Accuracy after query 27: 0.2896
Accuracy after query 28: 0.2849
Accuracy after query 29: 0.2960
Accuracy after query 30: 0.2906
Accuracy after query 31: 0.3021
Accuracy after query 32: 0.2970
Accuracy after query 33: 0.2936
Accuracy after query 34: 0.2999
Accuracy after query 35: 0.3011
Accuracy after query 36: 0.2985
Accuracy after query 37: 0.2984
Accuracy after query 38: 0.2902
Accuracy after query 39: 0.3017
Accuracy after query 40: 0.2956
Accuracy after query 41: 0.3050
Accuracy after query 42: 0.3048
Accuracy after query 43: 0.3062
Accuracy after query 44: 0.3060
Accuracy after query 45: 0.3142
Accuracy after query 46: 0.3123
Accuracy after query 47: 0.3081
Accuracy after query 48: 0.3153
Accuracy after query 49: 0.3121
Accuracy after query 50: 0.3146
Accuracy after query 51: 0.3135
Accuracy after query 52: 0.3089
Accuracy after query 53: 0.3200
Accuracy

Accuracy after query 83: 0.2969
Accuracy after query 84: 0.3040
Accuracy after query 85: 0.3105
Accuracy after query 86: 0.3078
Accuracy after query 87: 0.3091
Accuracy after query 88: 0.3030
Accuracy after query 89: 0.3010
Accuracy after query 90: 0.3060
Accuracy after query 91: 0.3073
Accuracy after query 92: 0.3073
Accuracy after query 93: 0.3099
Accuracy after query 94: 0.3112
Accuracy after query 95: 0.3098
Accuracy after query 96: 0.3144
Accuracy after query 97: 0.3104
Accuracy after query 98: 0.3119
Accuracy after query 1: 0.1896
Accuracy after query 2: 0.1940
Accuracy after query 3: 0.2039
Accuracy after query 4: 0.1947
Accuracy after query 5: 0.1948
Accuracy after query 6: 0.2024
Accuracy after query 7: 0.2002
Accuracy after query 8: 0.2076
Accuracy after query 9: 0.2219
Accuracy after query 10: 0.2174
Accuracy after query 11: 0.2255
Accuracy after query 12: 0.2175
Accuracy after query 13: 0.2310
Accuracy after query 14: 0.2317
Accuracy after query 15: 0.2412
Accuracy after qu

Accuracy after query 45: 0.2776
Accuracy after query 46: 0.2804
Accuracy after query 47: 0.2739
Accuracy after query 48: 0.2740
Accuracy after query 49: 0.2754
Accuracy after query 50: 0.2815
Accuracy after query 51: 0.2747
Accuracy after query 52: 0.2794
Accuracy after query 53: 0.2862
Accuracy after query 54: 0.2826
Accuracy after query 55: 0.2796
Accuracy after query 56: 0.2877
Accuracy after query 57: 0.2858
Accuracy after query 58: 0.2877
Accuracy after query 59: 0.2759
Accuracy after query 60: 0.2859
Accuracy after query 61: 0.2892
Accuracy after query 62: 0.2839
Accuracy after query 63: 0.2853
Accuracy after query 64: 0.2856
Accuracy after query 65: 0.2860
Accuracy after query 66: 0.2861
Accuracy after query 67: 0.2844
Accuracy after query 68: 0.2896
Accuracy after query 69: 0.2967
Accuracy after query 70: 0.2931
Accuracy after query 71: 0.3061
Accuracy after query 72: 0.3006
Accuracy after query 73: 0.2993
Accuracy after query 74: 0.3045
Accuracy after query 75: 0.3004
Accuracy

Accuracy after query 7: 0.1990
Accuracy after query 8: 0.2151
Accuracy after query 9: 0.2126
Accuracy after query 10: 0.2115
Accuracy after query 11: 0.2243
Accuracy after query 12: 0.2194
Accuracy after query 13: 0.2276
Accuracy after query 14: 0.2235
Accuracy after query 15: 0.2341
Accuracy after query 16: 0.2346
Accuracy after query 17: 0.2357
Accuracy after query 18: 0.2409
Accuracy after query 19: 0.2444
Accuracy after query 20: 0.2478
Accuracy after query 21: 0.2576
Accuracy after query 22: 0.2603
Accuracy after query 23: 0.2624
Accuracy after query 24: 0.2571
Accuracy after query 25: 0.2555
Accuracy after query 26: 0.2584
Accuracy after query 27: 0.2618
Accuracy after query 28: 0.2710
Accuracy after query 29: 0.2830
Accuracy after query 30: 0.2745
Accuracy after query 31: 0.2791
Accuracy after query 32: 0.2799
Accuracy after query 33: 0.2873
Accuracy after query 34: 0.2778
Accuracy after query 35: 0.2701
Accuracy after query 36: 0.2943
Accuracy after query 37: 0.2892
Accuracy af